### 비정형 데이터를 이용한 몽고디비
- not only SQL(NoSql)
- 정해진 규칙이 없음을 의미함 (스키마 없다)
- 관계형 DB가 아니기 때문에 join 등 불가능
- key : value 형식으로 데이터를 담는다

#### 저장하는 형식
- 다른 언어들과 불리는 이름이 조금 다르다
- Database - Collection(table) - Document(row)

In [1]:
import pymongo

### 1. 접속

In [2]:
connection = pymongo.MongoClient("localhost", 27017)

In [3]:
# default 값으로 들어갈거다 "localhost" 27017
connection = pymongo.MongoClient()

#### a) Database들을 보고싶을때

In [4]:
connection.list_database_names()

['BigData',
 'admin',
 'config',
 'local',
 'mydatabase',
 'test',
 'weatehr',
 '쥬피터에서만든db']

#### b) Database 생성 및 선택
- 존재하는 데이터베이스는 .을 이용해 속성 처럼 부를수있다
- 새로 생성할땐 []을 이용해서 key value로 새로운 데이터 베이스 생성
    > 새로 생성시 이미 존재하는 db이면 해당 db가 선택된다
- pymongo통해 db 생성시 => database is not created until it gets content!

In [9]:
BigData_db = connection.BigData
BigData_db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'BigData')

In [13]:
db = connection['쥬피터에서만든db'] 
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), '쥬피터에서만든db')

#### c) Collection들을 보고싶을때

In [15]:
print(BigData_db.list_collection_names())

['sleepdata']


#### d) Collection 생성 및 선택
- Database 생성 및 선택과 동일한 방식으로
- pymongo통해 db 생성시 => database is not created until it gets content!

In [16]:
collection = db['쥬피터에서만든collection']
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), '쥬피터에서만든db'), '쥬피터에서만든collection')

### 2. CRUD (create, read, update, delete)


#### a) 데이터 삽입
- insert_one(dict형식) : 한개의 document 
- insert_many([dict,dict,,,]) : 여러개의 document
- 구체적인 schema가 없기 때문에 형식이 자유롭다 넣고 싶은대로 넣어라 => 다만 dict(json)형식만 유지해서

In [57]:
# insert_one()
collection.insert_one({
    'author' : 'kanghee01',
    'subject' : 'mongodb',
    'int_value' : 10,
    'list_value' : [1,2,3]
})

In [19]:
# read 미리 맞만 보자
# documet를 한줄씩 출력하는 코드
for i in collection.find():
    print(i)

{'_id': ObjectId('61422c39b973c29ff223fc7f'), 'author': 'kanghee01', 'subject': 'mongodb', 'int_value': 10, 'list_value': [1, 2, 3]}


In [21]:
# insert_many()
collection.insert_many(
    [
        {
            'author' : 'kanghee02',
            'subject' : 'mongodb',
            'int_value' : 8,
            'dict_value' : {'a':1,'b':2}
        } ,
        {
            'author' : 'kanghee03',
            'subject' : 'mongodb',
            'int_value' : 8,
            'dict_value' : {'a':1,'b':2,'c':3}
        }
    ]
)

In [22]:
for i in collection.find():
    print(i)

{'_id': ObjectId('61422c39b973c29ff223fc7f'), 'author': 'kanghee01', 'subject': 'mongodb', 'int_value': 10, 'list_value': [1, 2, 3]}
{'_id': ObjectId('61422cfeb973c29ff223fc80'), 'author': 'kanghee02', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61422cfeb973c29ff223fc81'), 'author': 'kanghee03', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}


#### 주의! duplicate keyerror

In [24]:
document01 = {'dummy':'dummy value'}
collection.insert_one(document01)

In [34]:
# 변수로 저장되어 있는 document01을 동일하게 한번 더 넣으려고 하면 duplicate keyerror난다 
# 이렇게 사용하지 말자
collection.insert_one(document01.copy()) # 에러코드

DuplicateKeyError: E11000 duplicate key error collection: 쥬피터에서만든db.쥬피터에서만든collection index: _id_ dup key: { _id: ObjectId('61422d3bb973c29ff223fc82') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('61422d3bb973c29ff223fc82')}, 'errmsg': "E11000 duplicate key error collection: 쥬피터에서만든db.쥬피터에서만든collection index: _id_ dup key: { _id: ObjectId('61422d3bb973c29ff223fc82') }"}

#### b) 데이터 읽기
- find_one()
- find() => 반환타입이 Cursor다, 즉 쉽게말해 for문으로 document를 꺼내야한다
    - find({select쿼리} {프로젝션})
    - 비교문법
        - \$gt >
        - \$gte >=
        - \$lt <
        - \$lte <=
        - \$ne !=
        - \$eq ==
        - \$in in

In [37]:
# find_one 
collection.find_one() # 첫번째 document 꺼내온다

{'_id': ObjectId('61422c39b973c29ff223fc7f'),
 'author': 'kanghee01',
 'subject': 'mongodb',
 'int_value': 10,
 'list_value': [1, 2, 3]}

In [35]:
collection.find() # 타입에 주의하자

In [38]:
for i in collection.find():
    print(i)

{'_id': ObjectId('61422c39b973c29ff223fc7f'), 'author': 'kanghee01', 'subject': 'mongodb', 'int_value': 10, 'list_value': [1, 2, 3]}
{'_id': ObjectId('61422cfeb973c29ff223fc80'), 'author': 'kanghee02', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61422cfeb973c29ff223fc81'), 'author': 'kanghee03', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('61422d3bb973c29ff223fc82'), 'dummy': 'dummy value'}


In [40]:
# 위와 동일한 값이다
for i in collection.find({}):
    print(i)

{'_id': ObjectId('61422c39b973c29ff223fc7f'), 'author': 'kanghee01', 'subject': 'mongodb', 'int_value': 10, 'list_value': [1, 2, 3]}
{'_id': ObjectId('61422cfeb973c29ff223fc80'), 'author': 'kanghee02', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61422cfeb973c29ff223fc81'), 'author': 'kanghee03', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('61422d3bb973c29ff223fc82'), 'dummy': 'dummy value'}


In [41]:
# 실습을 위해 데이터를 더 집어넣어보자
collection.insert_many(
    [
        {
            'author' : 'kanghee04',
            'subject' : 'mongodb',
            'int_value' : 9,
            'dict_value' : {'a':1,'b':2}
        } ,
        {
            'author' : 'kanghee04',
            'subject' : 'mongodb',
            'int_value' : 10,
            'dict_value' : {'a':1,'b':2,'c':3}
        }
    ]
)

In [45]:
# find_one 쿼리
# select * from table wher author =='kanghee04'
for i in collection.find({'author':'kanghee04'}):
    print(i)

{'_id': ObjectId('61423285b973c29ff223fc83'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 9, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61423285b973c29ff223fc84'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 10, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}


In [46]:
# 비교문법을 사용한 쿼리문
for i in collection.find({'int_value':{'$gt':8}}):
    print(i)


{'_id': ObjectId('61422c39b973c29ff223fc7f'), 'author': 'kanghee01', 'subject': 'mongodb', 'int_value': 10, 'list_value': [1, 2, 3]}
{'_id': ObjectId('61423285b973c29ff223fc83'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 9, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61423285b973c29ff223fc84'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 10, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}


In [49]:
for i in collection.find({'int_value':{'$lte':8}}):
    print(i)


{'_id': ObjectId('61422cfeb973c29ff223fc80'), 'author': 'kanghee02', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61422cfeb973c29ff223fc81'), 'author': 'kanghee03', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}


In [50]:
for i in collection.find({'int_value':{'$in':[9,10]}}):
    print(i)

{'_id': ObjectId('61422c39b973c29ff223fc7f'), 'author': 'kanghee01', 'subject': 'mongodb', 'int_value': 10, 'list_value': [1, 2, 3]}
{'_id': ObjectId('61423285b973c29ff223fc83'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 9, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61423285b973c29ff223fc84'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 10, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}


In [54]:
# 두개 이상의 조건도 가능하다
for i in collection.find({'int_value':{'$in':[9,10]},'author':'kanghee04'}):
    print(i)

{'_id': ObjectId('61423285b973c29ff223fc83'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 9, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61423285b973c29ff223fc84'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 10, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}


In [52]:
# 프로젝션
# 원하는 fild명에 1을 준다
# 다만 _id는 무조건 나오기때문에 다시 설정 필요
for i in collection.find({'int_value':{'$in':[9,10]}},{'int_value':1}):
    print(i)

{'_id': ObjectId('61422c39b973c29ff223fc7f'), 'int_value': 10}
{'_id': ObjectId('61423285b973c29ff223fc83'), 'int_value': 9}
{'_id': ObjectId('61423285b973c29ff223fc84'), 'int_value': 10}


In [53]:
for i in collection.find({'int_value':{'$in':[9,10]}},{'int_value':1,'_id':0,'author':1}):
    print(i)

{'author': 'kanghee01', 'int_value': 10}
{'author': 'kanghee04', 'int_value': 9}
{'author': 'kanghee04', 'int_value': 10}


#### 추가적인 find옵션
- find().limit()
- find().sort()

In [68]:
# head와 비슷한 역할
for i in collection.find().limit(3):
    print(i)

{'_id': ObjectId('61422cfeb973c29ff223fc80'), 'author': 'kanghee02', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61422cfeb973c29ff223fc81'), 'author': 'kanghee03', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('61422d3bb973c29ff223fc82'), 'dummy': 'dummy value'}


In [72]:
# sort 1은 ascending =True
for i in collection.find().sort('int_value',1):
    print(i)

{'_id': ObjectId('61422d3bb973c29ff223fc82'), 'dummy': 'dummy value'}
{'_id': ObjectId('61422cfeb973c29ff223fc80'), 'author': 'kanghee02', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61422cfeb973c29ff223fc81'), 'author': 'kanghee03', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('6142350eb973c29ff223fc85'), 'author': 'kanghee01', 'subject': 'mysql', 'int_value': 10, 'list_value': [1, 2, 3]}
{'_id': ObjectId('61423285b973c29ff223fc83'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 100, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61423285b973c29ff223fc84'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 100, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}


In [73]:
# sort -1은 ascending =False
for i in collection.find().sort('int_value',-1):
    print(i)

{'_id': ObjectId('61423285b973c29ff223fc83'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 100, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61423285b973c29ff223fc84'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 100, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('6142350eb973c29ff223fc85'), 'author': 'kanghee01', 'subject': 'mysql', 'int_value': 10, 'list_value': [1, 2, 3]}
{'_id': ObjectId('61422cfeb973c29ff223fc80'), 'author': 'kanghee02', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61422cfeb973c29ff223fc81'), 'author': 'kanghee03', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('61422d3bb973c29ff223fc82'), 'dummy': 'dummy value'}


#### c) update
- update() 명령어는 depreciated 
- update_one({select쿼리}{update쿼리})
- update_many({select쿼리}{update쿼리})
#####  update 주로 사용하는 operator
- \$set : 필드값을 수정 => `{ $set: { <field1>: <value1>, ... } }`
- \$unset : 필들를 삭제 => `{ $unset: { <field1>: "", ... } }`

In [62]:
collection.update_one({'author':'kanghee01'},{'$set':{'subject':'mysql'}})
for i in collection.find():
    print(i)

{'_id': ObjectId('61422cfeb973c29ff223fc80'), 'author': 'kanghee02', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61422cfeb973c29ff223fc81'), 'author': 'kanghee03', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('61422d3bb973c29ff223fc82'), 'dummy': 'dummy value'}
{'_id': ObjectId('61423285b973c29ff223fc83'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 9, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61423285b973c29ff223fc84'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 10, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('6142350eb973c29ff223fc85'), 'author': 'kanghee01', 'subject': 'mysql', 'int_value': 10, 'list_value': [1, 2, 3]}


In [66]:
collection.update_many({'author':'kanghee04'},{'$set':{'int_value':100}})
for i in collection.find():
    print(i)

{'_id': ObjectId('61422cfeb973c29ff223fc80'), 'author': 'kanghee02', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61422cfeb973c29ff223fc81'), 'author': 'kanghee03', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('61422d3bb973c29ff223fc82'), 'dummy': 'dummy value'}
{'_id': ObjectId('61423285b973c29ff223fc83'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 100, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61423285b973c29ff223fc84'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 100, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('6142350eb973c29ff223fc85'), 'author': 'kanghee01', 'subject': 'mysql', 'int_value': 10, 'list_value': [1, 2, 3]}


In [76]:
# 동시에 여러개 수정도 가능하다
collection.update_many({'author':'kanghee01'},{'$set':{'subject':'psql','int_value':None}})
for i in collection.find():
    print(i)

{'_id': ObjectId('61422cfeb973c29ff223fc80'), 'author': 'kanghee02', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61422cfeb973c29ff223fc81'), 'author': 'kanghee03', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('61422d3bb973c29ff223fc82'), 'dummy': 'dummy value'}
{'_id': ObjectId('61423285b973c29ff223fc83'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 100, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61423285b973c29ff223fc84'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 100, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('6142350eb973c29ff223fc85'), 'author': 'kanghee01', 'subject': 'psql', 'int_value': None, 'list_value': [1, 2, 3]}


In [75]:
# 필드를 지워보자
# author가 kanghee03인 document의 subjec필드를 지워버렸다
collection.update_one({'author':'kanghee03'},{'$unset':{'subject':''}})
for i in collection.find():
    print(i)

{'_id': ObjectId('61422cfeb973c29ff223fc80'), 'author': 'kanghee02', 'subject': 'mongodb', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61422cfeb973c29ff223fc81'), 'author': 'kanghee03', 'int_value': 8, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('61422d3bb973c29ff223fc82'), 'dummy': 'dummy value'}
{'_id': ObjectId('61423285b973c29ff223fc83'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 100, 'dict_value': {'a': 1, 'b': 2}}
{'_id': ObjectId('61423285b973c29ff223fc84'), 'author': 'kanghee04', 'subject': 'mongodb', 'int_value': 100, 'dict_value': {'a': 1, 'b': 2, 'c': 3}}
{'_id': ObjectId('6142350eb973c29ff223fc85'), 'author': 'kanghee01', 'subject': 'mysql', 'int_value': 10, 'list_value': [1, 2, 3]}


#### d) delete
- delete_one({select쿼리})
- delete_many({select쿼리})


In [60]:
collection.delete_one({'subject':'mysql'})
for i in collection.fine():
    print(i)

In [ ]:
collection.delete_many({'int_value':8})
for i in collection.fine():
    print(i)

#### e) db 및 collection drop

In [79]:
connection.list_database_names()

['BigData',
 'admin',
 'config',
 'local',
 'mydatabase',
 'test',
 'weatehr',
 '쥬피터에서만든db']

In [81]:
# db 지우기
connection.drop_database('test')
connection.list_database_names()

['BigData', 'admin', 'config', 'local', 'mydatabase', 'weatehr', '쥬피터에서만든db']

In [82]:
drop_db = connection['mydatabase']
drop_db.list_collection_names()

['collection']

In [83]:
# collection 지우기
drop_collection = drop_db['collection']
drop_collection.drop()
drop_db.list_collection_names()

[]